In [24]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

In [25]:
!dir

 Volume in drive D has no label.
 Volume Serial Number is 3417-05D0

 Directory of d:\PythonTraining\Signate\【SOTA】アップル 引越し需要予測

2020/07/24  19:30    <DIR>          .
2020/07/24  19:30    <DIR>          ..
2020/07/24  19:04    <DIR>          .ipynb_checkpoints
2020/07/24  19:30            17,701 20200724.ipynb
2020/07/24  18:28            10,921 sample_submit.csv
2020/07/24  18:28             7,351 test.csv
2020/07/24  18:28            48,706 train.csv
               4 File(s)         84,679 bytes
               3 Dir(s)  958,200,905,728 bytes free


In [26]:
# read data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit = pd.read_csv('sample_submit.csv', header=None)

In [27]:
train

,datetime,y,client,close,price_am,price_pm
0,2010-07-01,17,0,0,-1,-1
1,2010-07-02,18,0,0,-1,-1
2,2010-07-03,20,0,0,-1,-1
3,2010-07-04,20,0,0,-1,-1
4,2010-07-05,14,0,0,-1,-1
...,...,...,...,...,...,...
2096,2016-03-27,94,1,0,5,4
2097,2016-03-28,86,1,0,4,4
2098,2016-03-29,98,1,0,4,4
2099,2016-03-30,99,1,0,5,4


In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2101 entries, 0 to 2100
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  2101 non-null   object
 1   y         2101 non-null   int64 
 2   client    2101 non-null   int64 
 3   close     2101 non-null   int64 
 4   price_am  2101 non-null   int64 
 5   price_pm  2101 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 98.6+ KB


In [29]:
train.describe(include='all')

,datetime,y,client,close,price_am,price_pm
count,2101,2101.000000,2101.000000,2101.000000,2101.000000,2101.000000
unique,2101,NaN,NaN,NaN,NaN,NaN
top,2014-04-11,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,34.096621,0.098049,0.012851,0.614469,0.390766
std,NaN,17.476234,0.297451,0.112658,1.127940,0.975456
min,NaN,0.000000,0.000000,0.000000,-1.000000,-1.000000
25%,NaN,21.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,32.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,45.000000,0.000000,0.000000,1.000000,1.000000


In [30]:
X = train[['client', 'close', 'price_am', 'price_pm']]

In [31]:
X['price_am'].value_counts()

 0    945
 1    587
 2    250
-1    211
 3     46
 4     33
 5     29
Name: price_am, dtype: int64

In [32]:
X['price_pm'].value_counts()

 0    1191
 1     496
-1     212
 2     125
 3      34
 4      30
 5      13
Name: price_pm, dtype: int64

In [33]:
train[(train['price_am'] == -1) & (train['price_pm'] != -1)]

,datetime,y,client,close,price_am,price_pm


In [34]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [35]:
y = train['y']

In [38]:
params = {'max_depth': list(range(2, 10, 2))}

# params = {'max_depth': list(range(2, 10, 2)),
#           'eta': [0.01, 0.1, 1.0],
#           'gamma': [0, 0.1],
#           'min_child_weight': [1, 2],
#           'nthread': [2, 4],
#           'n_estimators': list(range(50, 200, 50))}

xgb = XGBClassifier()

reg_cv = GridSearchCV(xgb, params, cv=5, return_train_score=True, verbose=1)
reg_cv.fit(X, y)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


D:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

In [ ]:
xgb = reg_cv.best_estimator_
score = xgb.score(X_test, y_test)
print('score: {}' .format(score))